In [5]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    # !wandb login
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/temporal')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Extra_and_Unused')
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/"

Running local


In [6]:
# from __future__ import print_function
# from __future__ import print_function
import os
import numpy as np

import datetime
import gc

from scipy import signal

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform

from Zamuro_DataLoader import SoundscapeData
from Models import Model
from Models import Encoder
from Models import Decoder
from Models import VectorQuantizer
from Models import VectorQuantizerEMA
from VQ_VAE_training_functions import TestModel, TrainModel # For VQ-VAE
# from PosAE_training_functions import posautoencoding_m1 as AE, TestModel, TrainModel # For PosAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

from datetime import timedelta

# wandb.login()

cuda


In [7]:
def featuring_autoencoders(dataset, date_format, model, len_features=None, save=True, identifier=None):
    training_loader = DataLoader(dataset, batch_size=1)
    iterator = iter(training_loader)
    testing = TestModel(model, iterator, device=torch.device("cuda"))

    training_recorder_list = []
    training_hour_list = []
    training_minute_list = []
    delete_samples = []
    training_path_samples = []
    training_samples_list_torch = []
    
    if len_features == None:
        len_features = len(training_loader)
    else:
        len_features = len_features
        
    batch = int(len_features*0.1)
    
    for id in range(len_features):
        if (id+1)% batch == 0:
            print(f"id: {id + 1} of {len_features}")
        try:
            originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
        except:
            print(f"error id: {id}")
            delete_samples.append(id)
            continue
        print(encodings.shape)
    #     encodings_size = encodings[0].shape
        encodings = encodings.to("cpu").detach()
        encodings = encodings.reshape(encodings.shape[0],
                                    encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
        training_samples_list_torch.append(encodings)
        label["recorder"]=np.repeat(label["recorder"][0][0],5)
#         training_recorder_list.append(label["recorder"])
#         training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
#         training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))


        path = np.asarray(path, dtype="U32")
        path = np.repeat(path, 5)
        indexer = ["_1","_2","_3","_4","_5"]
        indexer = np.asarray(indexer)
        # path = path.astype("float64")
        for i in range(len(path)):
            path[i] = label["recorder"][0] + "_" + path[i] + indexer[i]

        training_path_samples.append(path)
    
#     training_recorder_list = torch.cat(training_recorder_list,dim=0)
#     training_hour_list = torch.cat(training_hour_list,dim=0)
#     training_minute_list = torch.cat(training_minute_list,dim=0)
    training_samples_list_torch = torch.cat(training_samples_list_torch, dim=0)
    
    if save == True:
    
        if "filters" in dataset.kwargs.keys():
            for value in dataset.filters.values():
                date_format = f"{date_format}_{value}" 
        if identifier != None:
            date_format = f"{date_format}_{identifier}"
        else:
            pass

        torch.save(training_path_samples, f"temporal_zamuro/Features/VQ-VAE/VQ_test_path_samples_{date_format}.pth")
        torch.save(training_samples_list_torch, f"temporal_zamuro/Features/VQ-VAE/VQ_features_{date_format}.pth")
#         training_labels_list = {"recorder": training_recorder_list, "hour": training_hour_list, "minute": training_minute_list}
#         torch.save(training_labels_list, f"temporal_zamuro/Features/epochs_50/AE_labels_{date_format}.pth")
#         torch.save(delete_samples, f"temporal_zamuro/Features/epochs_50/AE_test_corrupted_samples_list_{date_format}.pth")
        
    else:
        pass

In [ ]:
# models = ["epoch_50_training.pth", "epoch_40_training.pth", "epoch_30_training.pth", "epoch_20_training.pth", "epoch_10_training.pth"]

date_format = "month_8_day_29_hour_12"
epochs = [10]
for i, epoch in enumerate(epochs):
    print(f"Running epoch: {epoch}")
    model_name = f"{root}/Zamuro/temporal_zamuro/models/model_VQ_batch_size_14_num_embeddings_256_embedding_dim_128__month_8_day_29_hour_12_training/epoch_{epoch}_training.pth"
    model = Model(num_hiddens=64, num_embeddings=256, embedding_dim=128, commitment_cost=0.25, decay=0.99).to(device)
    model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))

    import os
    folders = os.listdir(f"Complementary_Files/Audios_Zamuro/")

    for folder in folders:
        print(folder)
        fold = folder.split(".")[0]        
        filters = {"rain_FI": "NO"}
        dataset = SoundscapeData('media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/',
                             dataframe_path=f"Complementary_Files/Audios_Zamuro/{folder}",
                             audio_length=12, ext="wav",
                             win_length=1028, filters=filters)
        featuring_autoencoders(dataset, f"{date_format}_rain", model=model, len_features=None, save=True, identifier=f"{fold}_epoch_{epoch}")

Running epoch: 10
RZUD06.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torch.Size([5, 64, 128, 128])
torc

In [ ]:
import os
import shutil

# Ruta de la carpeta que contiene los archivos
input_folder = '/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics/Zamuro/temporal_zamuro/Features/VQ-VAE/'

# Ruta donde se crearán las nuevas carpetas
output_folder = '/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics/Zamuro/temporal_zamuro/Features/VQ-VAE//ordered'

# Listar todos los archivos en la carpeta de entrada
files = os.listdir(input_folder)

# Iterar sobre todos los archivos
for file in files:
    # Verificar si el archivo termina en '.pth'
    if file.endswith('.pth'):
        # Extraer el número del sufijo (el último número antes de .pth)
        suffix = file.split('_')[-1].split('.')[0]
        
        # Crear el nombre de la carpeta de destino basado en el sufijo
        target_folder = os.path.join(output_folder, suffix)
        
        # Crear la carpeta si no existe
        os.makedirs(target_folder, exist_ok=True)
        
        # Mover el archivo a la carpeta de destino
        shutil.move(os.path.join(input_folder, file), os.path.join(target_folder, file))

print("Archivos organizados correctamente.")

In [ ]:
import pandas as pd
audios_zamuro = pd.read_csv(f"{root}/Zamuro/Complementary_Files/zamuro_audios.csv")

In [ ]:
lista = list(set(audios_zamuro["field_number_PR"]))
lista.sort()

month = 7
day = 25
hour = 50
date_format = f"month_{month}_day_{day}_{hour}"

In [ ]:
X = torch.load(f"temporal_zamuro/Features/standarized_epochs_50/7/AE_features_{date_format}_rain_NO_{lista[0]}_epoch_7.pth",  map_location=torch.device('cpu'))
y = torch.load(f"temporal_zamuro/Features/standarized_epochs_50/7/AE_test_path_samples_{date_format}_rain_NO_{lista[0]}_epoch_7.pth",  map_location=torch.device('cpu'))
for i, csv in enumerate(lista[1::]):
    X_i = torch.load(f"temporal_zamuro/Features/standarized_epochs_50/7/AE_features_{date_format}_rain_NO_{csv}_epoch_7.pth",  map_location=torch.device('cpu'))
    y_i = torch.load(f"temporal_zamuro/Features/standarized_epochs_50/7/AE_test_path_samples_{date_format}_rain_NO_{csv}_epoch_7.pth",  map_location=torch.device('cpu'))
    X = torch.cat((X,X_i),0)
    y = y + y_i
torch.save(X, f"temporal_zamuro/Features/standarized_epochs_50/7/AE_features_Zamuro.pth")
torch.save(y, f"temporal_zamuro/Features/standarized_epochs_50/7/AE_test_path_samples_Zamuro.pth")